# Advanced RAG with temporal filters using LlamaIndex and KDB.AI vector store

##### Note: This example requires a KDB.AI endpoint and API key. Sign up for a free [KDB.AI account](https://kdb.ai/get-started).

> [KDB.AI](https://kdb.ai/) is a powerful knowledge-based vector database and search engine that allows you to build scalable, reliable AI applications, using real-time data, by providing advanced search, recommendation and personalization.

This example demonstrates how to use KDB.AI to run semantic search, summarization and analysis of financial regulations around some specific moment in time.

To access your end point and API keys, sign up to KDB.AI here.

To set up your development environment, follow the instructions on the KDB.AI pre-requisites page.

The following examples demonstrate some of the ways you can interact with KDB.AI through LlamaIndex.

## Install dependencies with Pip

In order to successfully run this sample, note the following steps depending on where you are running this notebook:

-***Run Locally / Private Environment:*** The [Setup](https://github.com/KxSystems/kdbai-samples/blob/main/README.md#setup) steps in the repository's `README.md` will guide you on prerequisites and how to run this with Jupyter.


-***Colab / Hosted Environment:*** Open this notebook in Colab and run through the cells.

In [ ]:
!pip install llama-index llama-index-embeddings-huggingface llama-index-llms-openai llama-index-readers-file llama-index-vector-stores-kdbai
!pip install kdbai_client pandas

## Import dependencies

In [31]:
from getpass import getpass
import re
import os
import shutil
import time
import urllib

import pandas as pd

from llama_index.core import (
    Settings,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
    VectorStoreIndex,
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.vector_stores.kdbai import KDBAIVectorStore

import kdbai_client as kdbai

OUTDIR = "pdf"
RESET = True


#### Set OpenAI API key and choose the LLM and Embedding model to use:

In [3]:
LLM = 'gpt-3.5-turbo'
# LLM = "gpt-4-turbo-preview"  # Expensive !!!

EMBEDDING = "sentence-transformers/all-mpnet-base-v2"   # Free but takes significant time to generate embeddings (~25 minutes for this notebook)
#EMBEDDING = "text-embedding-3-large"  # has cost but generates embeddings much faster (~1 minute for this notebook)

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key: ")

OpenAI API key: ··········


## Create KDB.AI session and table

In [4]:
# vector DB imports
import os
from getpass import getpass
import kdbai_client as kdbai
import time

##### Option 1. KDB.AI Cloud

To use KDB.AI Cloud, you will need two session details - a URL endpoint and an API key.
To get these you can sign up for free [here](https://trykdb.kx.com/kdbai/signup).

You can connect to a KDB.AI Cloud session using `kdbai.Session` and passing the session URL endpoint and API key details from your KDB.AI Cloud portal.

If the environment variables `KDBAI_ENDPOINTS` and `KDBAI_API_KEY` exist on your system containing your KDB.AI Cloud portal details, these variables will automatically be used to connect.
If these do not exist, it will prompt you to enter your KDB.AI Cloud portal session URL endpoint and API key details.

In [41]:
#Set up KDB.AI endpoing and API key
KDBAI_ENDPOINT = (
    os.environ["KDBAI_ENDPOINT"]
    if "KDBAI_ENDPOINT" in os.environ
    else input("KDB.AI endpoint: ")
)
KDBAI_API_KEY = (
    os.environ["KDBAI_API_KEY"]
    if "KDBAI_API_KEY" in os.environ
    else getpass("KDB.AI API key: ")
)

KDB.AI endpoint: your_kdbai_endpoint
KDB.AI API key: ··········


In [22]:
#connect to KDB.AI
session = kdbai.Session(api_key=KDBAI_API_KEY, endpoint=KDBAI_ENDPOINT)

##### Option 2. KDB.AI Server

To use KDB.AI Server, you will need download and run your own container.
To do this, you will first need to sign up for free [here](https://trykdb.kx.com/kdbaiserver/signup/).

You will receive an email with the required license file and bearer token needed to download your instance.
Follow instructions in the signup email to get your session up and running.

Once the [setup steps](https://code.kx.com/kdbai/gettingStarted/kdb-ai-server-setup.html) are complete you can then connect to your KDB.AI Server session using `kdbai.Session` and passing your local endpoint.

In [ ]:
session = kdbai.Session()

### Create the schema for your KDB.AI table

***!!! Note:*** The 'dims' parameter in the embedding column must reflect the output dimensions of the embedding model you choose.

- Hugging Face 'sentence-transformers/all-mpnet-base-v2' outputs 768 dimensions

- OpenAI 'text-embedding-3-large' outputs 3072 dimensions.

In [23]:

schema = dict(
    columns=[
        dict(name="document_id", pytype="bytes"),
        dict(name="text", pytype="bytes"),
        dict(
            name="embedding",
            vectorIndex=dict(type="flat", metric="L2", dims=768),
        ),
        dict(name="title", pytype="bytes"),
        dict(name="publication_date", pytype="datetime64[ns]"),
    ]
)

In [26]:
KDBAI_TABLE_NAME = "reports"

# First ensure the table does not already exist
if KDBAI_TABLE_NAME in session.list():
    session.table(KDBAI_TABLE_NAME).drop()

#Create the table
table = session.create_table(KDBAI_TABLE_NAME, schema)

## Financial reports urls and metadata

In [27]:
INPUT_URLS = [
    "https://www.govinfo.gov/content/pkg/PLAW-106publ102/pdf/PLAW-106publ102.pdf",
    "https://www.govinfo.gov/content/pkg/PLAW-111publ203/pdf/PLAW-111publ203.pdf",
]

METADATA = {
    "pdf/PLAW-106publ102.pdf": {
        "title": "GRAMM–LEACH–BLILEY ACT, 1999",
        "publication_date": pd.to_datetime("1999-11-12"),
    },
    "pdf/PLAW-111publ203.pdf": {
        "title": "DODD-FRANK WALL STREET REFORM AND CONSUMER PROTECTION ACT, 2010",
        "publication_date": pd.to_datetime("2010-07-21"),
    },
}

## Download PDF files locally

In [28]:
%%time

CHUNK_SIZE = 512 * 1024


def download_file(url):
    print("Downloading %s..." % url)
    out = os.path.join(OUTDIR, os.path.basename(url))
    try:
        response = urllib.request.urlopen(url)
    except urllib.error.URLError as e:
        logging.exception("Failed to download %s !" % url)
    else:
        with open(out, "wb") as f:
            while True:
                chunk = response.read(CHUNK_SIZE)
                if chunk:
                    f.write(chunk)
                else:
                    break
    return out


if RESET:
    if os.path.exists(OUTDIR):
        shutil.rmtree(OUTDIR)
    os.mkdir(OUTDIR)

    local_files = [download_file(x) for x in INPUT_URLS]
    local_files[:10]

CPU times: user 39.2 ms, sys: 11.8 ms, total: 51.1 ms
Wall time: 823 ms


## Load local PDF files with LlamaIndex

In [29]:
%%time


def get_metadata(filepath):
    return METADATA[filepath]


documents = SimpleDirectoryReader(
    input_files=local_files,
    file_metadata=get_metadata,
)

docs = documents.load_data()
len(docs)

CPU times: user 12.6 s, sys: 61.5 ms, total: 12.7 s
Wall time: 12.9 s


994

## Setup LlamaIndex RAG pipeline using KDB.AI vector store

In [32]:
%%time

### Choose the embedding method
embed_model = HuggingFaceEmbedding(model_name=EMBEDDING) #Free but takes ~25 minutes when running in Colab
#embed_model = OpenAIEmbedding(model=EMBEDDING) #Has cost but takes ~1 minute when running in Colab


llm = OpenAI(temperature=0, model=LLM)
vector_store = KDBAIVectorStore(table)
service_context = ServiceContext.from_defaults(
    embed_model=embed_model, llm=llm
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    docs,
    service_context=service_context,
    storage_context=storage_context,
    transformations=[SentenceSplitter(chunk_size=2048, chunk_overlap=0)],
)

<timed exec>:8: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.


CPU times: user 23.5 s, sys: 394 ms, total: 23.9 s
Wall time: 48.9 s


## Setup the LlamaIndex Query Engine

In [33]:
%%time

# Using gpt-3.5-turbo, the 16k tokens context size can only fit around 15 pages of document.
# Using gpt-4-turbo-preview, the 128k tokens context size can take 100 pages.
K = 15

query_engine = index.as_query_engine(
    similarity_top_k=K,
    filter=[("<", "publication_date", "2008-09-15")],
    sort_by="publication_date",
)

CPU times: user 60.8 ms, sys: 5.83 ms, total: 66.6 ms
Wall time: 67.6 ms


## Before the 2008 crisis

In [34]:
%%time

result = query_engine.query(
    """
What was the main financial regulation in the US before the 2008 financial crisis ?
"""
)
print(result.response)

The main financial regulation in the US before the 2008 financial crisis was the Dodd-Frank Wall Street Reform and Consumer Protection Act of 2010.
CPU times: user 410 ms, sys: 8.35 ms, total: 418 ms
Wall time: 6.66 s


In [35]:
%%time

result = query_engine.query(
    """
Is the Gramm-Leach-Bliley Act of 1999 enough to prevent the 2008 crisis. Search the document and explain its strenghts and weaknesses to regulate the US stock market.
"""
)
print(result.response)

The Gramm-Leach-Bliley Act of 1999 sought to enhance competition in the financial services industry by allowing connections between banks, securities firms, and insurance companies. Its effectiveness is evident in setting standards for these connections and streamlining supervision of bank holding companies. Yet, potential drawbacks could stem from challenges related to resource concentration, conflicts of interest, and unsound banking practices. While the Act addressed some concerns, it may not have comprehensively addressed all the factors contributing to the 2008 financial crisis, such as inadequate oversight of specific financial products and institutions pivotal in the crisis.
CPU times: user 355 ms, sys: 16.5 ms, total: 371 ms
Wall time: 15.2 s


## After the 2008 crisis

In [36]:
%%time

# Using gpt-3.5-turbo, the 16k tokens context size can only fit around 15 pages of document.
# Using gpt-4-turbo-preview, the 128k tokens context size can take 100 pages.
K = 15

query_engine = index.as_query_engine(
    similarity_top_k=K,
    filter=[(">=", "publication_date", "2008-09-15")],
    sort_by="publication_date",
)

CPU times: user 471 µs, sys: 32 µs, total: 503 µs
Wall time: 514 µs


In [37]:
%%time

result = query_engine.query(
    """
What happened on the 15th of September 2008 ? Answer from your own knowledge only.
"""
)
print(result.response)

On the 15th of September 2008, Lehman Brothers filed for bankruptcy, which marked one of the largest bankruptcies in U.S. history and had significant repercussions on the global financial system.
CPU times: user 363 ms, sys: 11.9 ms, total: 375 ms
Wall time: 7.56 s


In [38]:
%%time

result = query_engine.query(
    """
What was the new US financial regulation enacted after the 2008 crisis to increase the market regulation and to improve consumer sentiment ?
"""
)
print(result.response)

The Dodd-Frank Wall Street Reform and Consumer Protection Act of 2010 was enacted as a new US financial regulation after the 2008 crisis to increase market regulation and improve consumer sentiment.
CPU times: user 341 ms, sys: 9.8 ms, total: 351 ms
Wall time: 9.44 s


## In depth analysis

In [39]:
%%time

# Using gpt-3.5-turbo, the 16k tokens context size can only fit around 15 pages of document.
# Using gpt-4-turbo-preview, the 128k tokens context size can take 100 pages.
K = 20

query_engine = index.as_query_engine(
    similarity_top_k=K, sort_by="publication_date"
)

CPU times: user 0 ns, sys: 396 µs, total: 396 µs
Wall time: 403 µs


In [40]:
%%time

result = query_engine.query(
    """
Analyse the US financial regulations before and after the 2008 crisis and produce a report of all related arguments to explain what happened, and to ensure that does not happen again.
Use both the provided context and your own knowledge but do mention explicitely which one you use.
"""
)
print(result.response)

The Dodd-Frank Wall Street Reform and Consumer Protection Act of 2010 was a response to the 2008 financial crisis, aiming to strengthen financial regulations. Before the crisis, regulatory oversight was criticized for being insufficient in monitoring large financial institutions, leading to unchecked risky behavior. The Act introduced measures to mitigate risks, such as restrictions on mergers and acquisitions and limits on certain financial activities. It also established the Office of Financial Research and the Financial Stability Oversight Council to enhance monitoring and address systemic risks.

To prevent future crises, the Act focused on increasing transparency, strengthening oversight, and enhancing consumer protection. By implementing stricter regulations and oversight mechanisms, it aimed to prevent excessive risk-taking and promote financial stability.

In summary, the Dodd-Frank Act significantly revamped financial regulations in the US post-2008 crisis to address regulator